## Importing Libraries

In [213]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, chi2
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.layers import Dense, Flatten
from keras.models import Sequential
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Data Cleaning and Analysis

In [214]:
path = "/Users/parvthakkar/Desktop/NEU MSDS/Capstone/Project_Data/vitaldb-a-high-fidelity-multi-parameter-vital-signs-database-in-surgical-patients-1.0.0/"

data = pd.read_csv(path + "clinical_data.csv")
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6388 entries, 0 to 6387
Data columns (total 74 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   caseid               6388 non-null   int64  
 1   subjectid            6388 non-null   int64  
 2   casestart            6388 non-null   int64  
 3   caseend              6388 non-null   int64  
 4   anestart             6388 non-null   int64  
 5   aneend               6388 non-null   float64
 6   opstart              6388 non-null   int64  
 7   opend                6388 non-null   int64  
 8   adm                  6388 non-null   int64  
 9   dis                  6388 non-null   int64  
 10  icu_days             6388 non-null   int64  
 11  death_inhosp         6388 non-null   int64  
 12  age                  6388 non-null   object 
 13  sex                  6388 non-null   object 
 14  height               6388 non-null   float64
 15  weight               6388 non-null   f

,caseid,subjectid,casestart,caseend,anestart,aneend,opstart,opend,adm,dis,...,intraop_colloid,intraop_ppf,intraop_mdz,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca
0,1,5955,0,11542,-552,10848.0,1668,10368,-236220,627780,...,0,120,0.0,100,70,0,10,0,0,0
1,2,2487,0,15741,-1039,14921.0,1721,14621,-221160,1506840,...,0,150,0.0,0,100,0,20,0,0,0
2,3,2861,0,4394,-590,4210.0,1090,3010,-218640,40560,...,0,0,0.0,0,50,0,0,0,0,0
3,4,1903,0,20990,-778,20222.0,2522,17822,-201120,576480,...,0,80,0.0,100,100,0,50,0,0,0
4,5,4416,0,21531,-1009,22391.0,2591,20291,-67560,3734040,...,0,0,0.0,0,160,0,10,900,0,2100


In [215]:
# Dropping columns that are not useful

drop_columns = ['subjectid', 'casestart', 'caseend', 'anestart', 
                'aneend', 'opstart', 'opend', 'adm', 'dis', 'dltubesize', 
                'lmasize', 'aline2', 'cline1', 'cline2']

data = data.drop(drop_columns, axis=1)

print(len(data['dx'].unique()))
data.shape

1038


(6388, 60)

### Cleaning Age

In [216]:
# Changing age to float

data['age'] = data['age'].replace(['>89'], '89')

data['age'] = pd.to_numeric(data['age'])
df = data['age'].value_counts().rename_axis('Age').reset_index(name='Count').sort_values(['Age'], ascending=False)

df.tail(10)

data = data[data.age > 1]

data.shape

(6378, 60)

In [217]:
# Chnaging Age to rasnges for encoding

df = data['age'].value_counts().rename_axis('Age').reset_index(name='Count').sort_values(['Age'], ascending=False)
df['AgeRange'] = pd.cut(df['Age'], 5)
print(df['AgeRange'].unique())
# df[''].groupby(['AgeRange'], as_index=False).count().sort_values('AgeRange', ascending=True)

data['age'] = np.where((data['age'] >= 4) & (data['age'] < 22), 0, data['age'])
data['age'] = np.where((data['age'] >= 22) & (data['age'] < 39), 1, data['age'])
data['age'] = np.where((data['age'] >= 39) & (data['age'] < 56), 2, data['age'])
data['age'] = np.where((data['age'] >= 56) & (data['age'] < 73), 3, data['age'])
data['age'] = np.where((data['age'] >= 73) & (data['age'] <= 89), 4, data['age'])

# data.loc[(data['age'] >= 22 and data['age'] < 39), 'age'] = 1
# data.loc[(data['age'] >= 39 and data['age'] < 56), 'age'] = 2
# data.loc[(data['age'] >= 56 and data['age'] < 73), 'age'] = 3
# data.loc[(data['age'] >= 73 and data['age'] <= 89), 'age'] = 4

data['age'].value_counts()

[(72.2, 89.0], (55.4, 72.2], (38.6, 55.4], (21.8, 38.6], (4.916, 21.8]]
Categories (5, interval[float64, right]): [(4.916, 21.8] < (21.8, 38.6] < (38.6, 55.4] < (55.4, 72.2] < (72.2, 89.0]]


3.0    2835
2.0    1772
4.0    1018
1.0     636
0.0     117
Name: age, dtype: int64

### Cleaning Diagnosis Columns

In [218]:
# Selecting Relevant Columns

iag_columns = ['age', 'sex', 'height', 'weight', 'bmi', 'asa', 'preop_htn', 'preop_dm', 
                'preop_ecg', 'preop_pft', 'preop_hb', 'preop_plt', 'preop_pt', 'preop_aptt',
                'preop_na', 'preop_k', 'preop_gluc', 'preop_alb', 'preop_ast',
                'preop_alt', 'preop_bun', 'preop_cr', 'preop_ph', 'preop_hco3',
                'preop_be', 'preop_pao2', 'preop_paco2', 'preop_sao2', 'dx', 'optype']

data = data[diag_columns]

In [219]:
len(data['dx'].unique())

1034

In [220]:
# Removing left and right from diagnosis as the next steps would be the same

data['dx'] = data['dx'].str.replace('left', '')
data['dx'] = data['dx'].str.replace('right', '')

data[data['optype'] == "Breast"]['dx'].value_counts()

Infiltrating ductal carcinoma of breast                                   142
Breast cancer                                                             124
Breast cancer unspecified side                                             40
Ductal carcinoma in situ of breast                                         39
Ductal carcinoma in situ of breast unspecified side                        22
Infiltrating ductal carcinoma of breast unspecified side                    9
Fibroadenoma of breast                                                      7
Malignant neoplasm of breast                                                6
Mucinous carcinoma of breast                                                6
Lymphadenopathy                                                             3
Intraductal carcinoma in situ of breast                                     3
Infiltrating lobular carcinoma of breast                                    3
Benign neoplasm of breast                                       

In [221]:
# Removing spaces and \n to avoid having multiple same name categories

data['dx'] = data['dx'].str.strip()
for i in data.columns:
    if data[i].dtype == "object":
        data[i] = data[i].str.strip()

len(data['dx'].unique())

842

In [222]:
# Removing Diagnosis having count 9 as the model would not accurately work for those

df = data['dx'].value_counts().rename_axis('dx').reset_index(name='Count').sort_values(['Count'], ascending=False)
print(df.shape)
one_list = list(df[df['Count'] <= 9]['dx'])
len(one_list)
data = data[~data['dx'].isin(one_list)]
data['dx'] = data['dx'].str.lower()

len(data['dx'].unique())

(842, 2)


97

In [223]:
op_list = list(data['optype'].value_counts().index)

print(op_list)
dx_op_list = []

ind_op_list = ['Colorectal', 'Breast', 'Biliary/Pancreas', 'Stomach', 'Major resection', 'Minor resection', 'Transplantation', 'Thyroid']

for op in ind_op_list:
    dx_op_list.append(list(data[data['optype'] == op]['dx'].unique()))
    # print(data[data['optype'] == op]['dx'].str.lower().value_counts())



['Colorectal', 'Biliary/Pancreas', 'Stomach', 'Major resection', 'Minor resection', 'Others', 'Breast', 'Transplantation', 'Hepatic', 'Thyroid', 'Vascular']


In [224]:
data[data['optype'] == "Others"].shape

(382, 30)

In [225]:
# Changing Others category which are falsely marked

for ind, row in data.iterrows():
    for i in range(len(dx_op_list)):
        if row['dx'] in dx_op_list[i]:
            # print(row['optype'])
            data['optype'][ind] = ind_op_list[i]
            # print(row['optype'])
            break

/var/folders/mp/xdrs4ffj5k7gc2rwyg0_mxrm0000gn/T/ipykernel_16112/1995343892.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['optype'][ind] = ind_op_list[i]


In [226]:
data[data['optype'] == "Others"].shape

(85, 30)

## Missing Values

### Lab Data Analysis

In [227]:
data.isnull().sum().sum()

30647

In [228]:
path = "/Users/parvthakkar/Desktop/NEU MSDS/Capstone/Project_Data/vitaldb-a-high-fidelity-multi-parameter-vital-signs-database-in-surgical-patients-1.0.0/"

lab_data = pd.read_csv(path + "lab_data.csv")

lab_data = lab_data.drop(['dt'], axis=1)
lab_data = lab_data.groupby(['caseid', 'name'], as_index=False)['result'].mean()

lab_data = lab_data.pivot(index='caseid', columns='name', values='result')
lab_data.columns = ['preop_' + i for i in lab_data.columns]

preop_lab_values = ['preop_htn', 'preop_dm', 'preop_ecg', 'preop_pft', 'preop_hb', 
                    'preop_plt', 'preop_pt', 'preop_aptt', 'preop_na', 'preop_k', 
                    'preop_gluc', 'preop_alb', 'preop_ast', 'preop_alt', 'preop_bun', 
                    'preop_cr', 'preop_ph', 'preop_hco3', 'preop_be', 'preop_pao2', 
                    'preop_paco2', 'preop_sao2']

preop_lab_values = [i for i in lab_data.columns if i in preop_lab_values]
not_preop_lab_values = [i for i in lab_data.columns if i not in preop_lab_values]


not_preop_lab_data = lab_data[not_preop_lab_values]

# print(not_preop_lab_data.info())

add_list = [i for i in not_preop_lab_data.columns if not_preop_lab_data[i].isnull().sum() < 1000]

# print(len(add_list))
not_preop_lab_data.head()

for i in add_list:
    preop_lab_values.append(i)

preop_lab_data = lab_data[preop_lab_values]



data = data.combine_first(preop_lab_data)

data.shape

(6236, 36)

## One Hot Encoding

In [229]:
# Performing one hot encoding as the data has nominal categorical variables

one = OneHotEncoder(handle_unknown='ignore', drop='first')

object_list = []

for i in data.columns:
    if data[i].dtype == "object" and i != 'dx' and i != 'optype':
        object_list.append(i)

print(object_list)

object_list = object_list[:len(object_list)-1]
print(object_list)

data_df = pd.get_dummies(data[object_list], drop_first=True)

data = data.join(data_df)

data = data.drop(['sex', 'preop_ecg', 'preop_pft'], axis=1)

data.columns


['preop_ecg', 'preop_pft', 'sex']
['preop_ecg', 'preop_pft']


Index(['age', 'asa', 'bmi', 'dx', 'height', 'optype', 'preop_alb', 'preop_alt',
       'preop_aptt', 'preop_ast', 'preop_be', 'preop_bun', 'preop_cl',
       'preop_cr', 'preop_dm', 'preop_gfr', 'preop_gluc', 'preop_hb',
       'preop_hco3', 'preop_hct', 'preop_htn', 'preop_k', 'preop_na',
       'preop_paco2', 'preop_pao2', 'preop_ph', 'preop_plt', 'preop_pt',
       'preop_sao2', 'preop_tbil', 'preop_tprot', 'preop_wbc', 'weight',
       'preop_ecg_1st degree A-V block with Premature atrial complexes',
       'preop_ecg_1st degree A-V block with Premature supraventricular complexes, Left bundle branch block',
       'preop_ecg_1st degree A-V block, Left bundle branch block',
       'preop_ecg_AV sequential or dual chamber electronic pacemaker',
       'preop_ecg_Atrial fibrillation',
       'preop_ecg_Atrial fibrillation with premature ventricular or aberrantly conducted complexes',
       'preop_ecg_Atrial fibrillation with premature ventricular, Incomplete left bundle block',
     

### Encoding Diagnosis

In [230]:
i = list(range(0, 1034))

dx_list = list(data['dx'].value_counts().index)

dx_dic = dict(zip(dx_list, i))

data['dx'] = data['dx'].map(dx_dic)

data['dx'].value_counts()

0.0     364
1.0     280
2.0     265
3.0     247
4.0     207
       ... 
92.0     10
93.0     10
94.0     10
95.0     10
96.0     10
Name: dx, Length: 97, dtype: int64

## Missing Data Handling

In [231]:
data = data[data['optype'].notna()]
data['optype'].isnull().sum()

0

In [232]:
# Replacing values with median tested for mean and random_sampling mean performs better

for i in data.columns:
    if data[i].isnull().sum() > 1000:
        data = data.drop(i, axis=1)

missing_features = []

for i in data.columns:
    if data[i].isnull().sum():
        missing_features.append(i)

# print(missing_features)

for i in missing_features:
    data[i] = data[i].fillna(data[i].median())

## Splitting the Data

In [233]:
# Splitting the data based on operation type

# print(op_list)

for op in op_list:
    print(len(data[data['optype'] == op]['dx'].value_counts()))
    print((data[data['optype'] == op]).shape)

34
(2212, 56)
20
(616, 56)
4
(192, 56)
16
(805, 56)
6
(135, 56)
5
(85, 56)
5
(400, 56)
4
(162, 56)
0
(0, 56)
3
(242, 56)
1
(9, 56)


In [234]:
split_data = []

for i in range(len(ind_op_list)):
    split_data.append(data[data['optype'] == ind_op_list[i]])

print(len(split_data))
for i in split_data:
    print(i.shape)

# Removing the operation type column as it will not be known for patient diagnosis
for i in range(len(split_data)):
    split_data[i] = split_data[i].drop('optype', axis=1)

8
(2212, 56)
(400, 56)
(616, 56)
(192, 56)
(805, 56)
(135, 56)
(162, 56)
(242, 56)


## Machine Learning Models

In [235]:
# def ann_model(target=10):

#     nn_model = Sequential()
#     nn_model.add(Dense(units=1258, input_dim=45,
#                  kernel_initializer='uniform', activation='relu'))
#     nn_model.add(Flatten())
#     nn_model.add(
#         Dense(units=1000, kernel_initializer='uniform', activation='relu'))
#     nn_model.add(
#         Dense(units=800, kernel_initializer='uniform', activation='relu'))
#     nn_model.add(
#         Dense(units=500, kernel_initializer='uniform', activation='relu'))
#     nn_model.add(
#         Dense(units=100, kernel_initializer='uniform', activation='relu'))
#     nn_model.add(
#         Dense(units=50, kernel_initializer='uniform', activation='relu'))
#     nn_model.add(
#         Dense(units=target, kernel_initializer='uniform', activation='softmax'))
#     nn_model.compile(loss='sparse_categorical_crossentropy',
#                      optimizer='adam', metrics=['acc'])

#     return nn_model

In [236]:
new_dx_dic = dict([(value, key) for key, value in dx_dic.items()])
new_dx_dic[0], new_dx_dic[9]

('early gastric cancer', 'breast cancer')

In [255]:
def rf_model(data, name):

    # Train Test Splits
    y = data['dx']
    x = data.drop(['dx'], axis=1)

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=42)

    # Constant Feature Check

    varThresh = VarianceThreshold(threshold=1)
    varThresh.fit(x_train)

    len(x_train.columns[varThresh.get_support()])

    # determine the mutual information

    mutual_info = mutual_info_classif(x_train, y_train)
    mutual_info = pd.Series(mutual_info)
    mutual_info.index = x_train.columns
    mv = mutual_info.sort_values(ascending=False)
    most_imp_feat1.append(mv.index[0])
    most_imp_feat2.append(mv.index[1])
    most_imp_feat3.append(mv.index[2])

    #let's plot the ordered mutual_info values per feature
    # Plotting the features having highest importance to lowest
    # mutual_info.sort_values(ascending=False)[:20].plot.bar(figsize=(20, 8))
    # plt.title("Feature Importances based on Mutual Information")
    # plt.xlabel("Features")
    # plt.ylabel("Mutual Information")
    # plt.savefig("graph" + name + ".png", bobx="tight", pad_inches=0)
    # plt.show()

    # Now we Will select the  top 20 important features
    sel_cols = SelectKBest(chi2, k=45)
    sel_cols.fit(x_train, y_train)
    x_test = x_test[x_train.columns[sel_cols.get_support()]]
    x_train = x_train[x_train.columns[sel_cols.get_support()]]

    # Random Forest Model
    rf = RandomForestClassifier(max_features = 5, n_estimators = 500)
    rf.fit(x_train, y_train)
    rfpred = rf.predict(x_test)
    rf_acc_list.append(accuracy_score(y_test, rfpred)*100)

    # # ANN Model
    # x_train, x_val, y_train, y_val = train_test_split(
    #     x_train, y_train, test_size=0.2, random_state=42)
    # model = ann_model(len(data['dx'].value_counts()))
    # print(len(data['dx'].value_counts()))
    # print(model.summary())
    # model.fit(x_train, y_train, epochs=150,
    #                     batch_size=100, validation_data=(x_val, y_val), verbose=0)
    # _, acc = model.evaluate(x_test, y_test)
    # ann_acc_list.append(acc)
    # print("The accuracy for Neural Network for " + name + " is ", acc)
    # print("The accuracy for model Random Forest for " + name + " is ", accuracy_score(y_test, rfpred))

    probs = rf.predict_proba(x_test)
    predictions = list(rf.classes_[np.argsort(probs)[:, :-3 - 1:-1]])
    # print(type(predictions))

    diag_pred = []
    
    for preds in predictions:
        # print(type(preds))
        # print(preds)
        temp = []
        for i in range(3):
            # print(i)
            # print(preds[int(i)])
            temp.append(new_dx_dic[preds[i]])
        diag_pred.append(temp)
        

    probabilities = [sorted(np.around(probas*100, 2), reverse=True)[:3] for probas in probs]

    d1, d2, d3 = [], [], []

    for d in diag_pred:
        d1.append(d[0])
        d2.append(d[1])
        d3.append(d[2])

    p1, p2, p3 = [], [], []

    for p in probabilities:
        p1.append(p[0])
        p2.append(p[1])
        p3.append(p[2])

    df = pd.DataFrame(list(zip(d1, p1, d2, p2, d3, p3)), columns=[
                      'Suggested Diagnosis 1', 'Confidence', 'Suggested Diagnosis 2', 'Confidence', 'Suggested Diagnosis 3', 'Confidence', ])

    pred_df.append(df)


rf_acc_list = []
ann_acc_list = []
most_imp_feat1 = []
most_imp_feat2 = []
most_imp_feat3 = []
pred_df = []

for i in range(len(split_data)):
    rf_model(split_data[i], ind_op_list[i])

unique_dx = []
for i in split_data:
    unique_dx.append(len(i['dx'].value_counts()))

most_imp_df = pd.DataFrame(list(zip(ind_op_list, most_imp_feat1, most_imp_feat2, most_imp_feat3)), columns=[
                           'Operation Type', 'Most important feature', '2nd Most important feature', '3rd Most important feature'])

rf_result_df = pd.DataFrame(list(zip(ind_op_list, rf_acc_list, unique_dx)), columns=[
                            'Operation Type', 'Diagnosis Accuracy', 'No. of Unique Diagnosis Present'])


In [238]:
rf_result_df

,Operation Type,Diagnosis Accuracy,No. of Unique Diagnosis Present
0,Colorectal,25.733634,34
1,Breast,31.250000,5
2,Biliary/Pancreas,21.774194,20
3,Stomach,74.358974,4
4,Major resection,37.888199,16
5,Minor resection,29.629630,6
6,Transplantation,57.575758,4
7,Thyroid,85.714286,3


In [239]:
most_imp_df

,Operation Type,Most important feature,2nd Most important feature,3rd Most important feature
0,Colorectal,preop_pft_Normal,preop_plt,asa
1,Breast,preop_ecg_Premature supraventricular and ventr...,preop_pft_Mild obstructive,preop_ecg_Atrial flutter with variable A-V block
2,Biliary/Pancreas,preop_alb,asa,preop_cr
3,Stomach,preop_cr,preop_bun,preop_hb
4,Major resection,height,age,preop_hb
5,Minor resection,age,height,preop_hct
6,Transplantation,preop_ecg_Normal Sinus Rhythm,age,preop_pft_Normal
7,Thyroid,preop_ecg_Atrial flutter with variable A-V block,age,preop_cr


In [254]:
pred_df[0].tail()

,Suggested Diagnosis 1,Confidence,Suggested Diagnosis 2,Confidence,Suggested Diagnosis 3,Confidence
438,hepatocellular carcinoma,22.4,early gastric cancer,16.8,varicose vein of lower limb,9.6
439,advanced gastric cancer,15.8,"colon cancer, sigmoid",14.0,rectal cancer,13.4
440,early gastric cancer,15.4,varicose vein of lower limb,12.0,rectal cancer,11.4
441,advanced gastric cancer,22.2,early gastric cancer,14.4,"colon cancer, sigmoid",14.2
442,early gastric cancer,15.4,advanced gastric cancer,13.2,rectal cancer,12.8


In [256]:
rf_result_df['Diagnosis Accuracy'].mean()

46.06271335006563